# Kears

- Keras의 개요
    - 파이썬으로 구현된 쉽고 간결한 딥러닝 라이브러리
    - 내부적으로 텐서플로우 엔진이 구동되지만 직관적인 API로 쉽게 딥러닝 실험을 할 수 있도록 지원함
    
- Keras의 주요 특징
    - 모듈화(Modularity) : 독립적인 모듈들을 조합하여 구현
    - 최소주의(Minialism) : 각 모듈은 짧고 간결
    - 쉬운 확장성 : 클래스나 함수로 모듈을 쉽게 추가할 수 있음
    - 파이썬 기반 : 별도의 설정이 필요없음
    
- keras를 이용한 실습 순서
    - 데이터셋 생성
        - 데이터로부터 학습용, 검증용 데이터셋 구분
        - 딥러닝 모델의 학습 및 검증을 할 수 있도록 케라스에서 요구하는 형태로 변환을 해야 함
    - 모델 구성
        - Sequence 모델을 생성한 후 필요한 Layer를 추가하여 구성
    - 모델 학습과정 설정
        - 손실 함수 및 최적화 방법 정의
        - 모델 컴파일
    - 모델 학습
        - 학습용 데이터를 이용하여 모델 학습
    - 학습과정 출력
        - 학습 시 학습용, 검증용 데이터셋의 손실 및 정확도 측정
    - 모델 평가
        - 검증용 데이터셋으로 학습한 모델을 평가
    - 새로운 데이터 사용하여 값을 예측

In [1]:
# Keras 기반의 회귀분석 예제
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import rc
import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import RMSprop
from mpl_toolkits.mplot3d import Axes3D

# 한글 처리를 위해 폰트 설정
rc('font', family='AppleGothic')
# 음수 부호가 깨지지 않도록 설정
plt.rcParams['axes.unicode_minus'] = False

Using TensorFlow backend.


In [2]:
# 보스턴 주택가격 예측 : 회귀분석
# 회귀분석 : 연속적인 값을 예측
# 예) 과거 기상 데이터를 입력하여 내일의 기온을 예측
# 보스턴 주택 가격 데이터셋
# 1970년 중반 보스턴 외곽 지역의 범죄율, 지방세율 등의 데이터로 주택 가격 예측
# 샘플 갯수 : 506개(학습용 404개, 검증용 102개)
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

In [3]:
# 학습용 404개, 검증용 102개, 13개의 변수
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
# 종속변수 : 주택의 가격(천달러 단위)
train_targets[0:10]

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4])

In [6]:
# 데이터 준비
# 상이한 스케일을 가진 값을 신경망에 입력하면 예측 정확도가 떨어질 수 있음
# 데이터 정규화 : 각 변수에 대해서 표준 편차가 1이 되도록 평균을 빼고 표준 편차로 나눈다.
# 평균값을 빼고
mean = train_data.mean(axis=0)
train_data -= mean
test_data -= mean

# 표준편차로 나눈다
std = train_data.std(axis=0)
train_data /= std
test_data /= std

In [7]:
train_data[0:5]

array([[-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
        -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
         1.14850044,  0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, -0.25683275, -1.21518188,
         1.89434613, -1.91036058,  1.24758524, -0.85646254, -0.34843254,
        -1.71818909,  0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , -0.25683275,  0.62864202,
        -1.82968811,  1.11048828, -1.18743907,  1.67588577,  1.5652875 ,
         0.78447637,  0.22061726, -1.30850006],
       [-0.40149354, -0.48361547, -0.86940196, -0.25683275, -0.3615597 ,
        -0.3245576 , -1.23667187,  1.10717989, -0.51114231, -1.094663  ,
         0.78447637,  0.44807713, -0.65292624],
       [-0.0056343 , -0.48361547,  1.0283258 , -0.25683275,  1.32861221,
         0.15364225,  0.69480801, -0.57857203,  1.67588577,  1.5652875 ,
         0.78447637,  0.3898823 ,  0.26349695]])

In [8]:
# 활성화 함수 : 입력데이터를 적절하게 처리하여 출력데이터를 만들어내는 함수
# input_data -> {activation function} -> output data
# 활성화 함수의 종류 : 탄젠트(thanh) 함수, 시그모이드(sigmoid) 함수, ReLU 함수
# 탄젠트 함수나 시그모이드 함수보다 학습시간이 빨라서 최근에는 ReLU(Rectified Linear Unit)가 많이 사용되고 있음
from keras import models
from keras import layers
# 64개의 노드를 가진 은닉층 2개의 신경망
def build_model():
    model = models.Sequential()
    # relu
    model.add(layers.Dense(64, activation='relu',
                          input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    
    # 출력층(1개의 노드, 하나의 값을 예측하는 회귀분석의 경우 활성화 함수가 없는 선형층을 사용함)
    model.add(layers.Dense(1))
    # mse 손실 함수를 사용하여 컴파일
    # mse : 평균 제곱 오차(mean squared error) - 예측값과 실제값의 거리의 제곱
    # mae : 평균 절대 오차, 예측값과 실제값의 거리의 절대값
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [9]:
# k-fold 교차 검증
# 샘플 갯수가 많지 않으므로 학습용 데이터의 크기가 매우 작아지게 됨(약 100개의 샘플)
# 그러면 샘플링할 때마다 검증용 데이터셋의 점수가 크게 달라지게 됨
# 이렇게 되면 모델의 신뢰성이 떨어질 수 있음
# 데이터건수가 적은 경우 k-fold(k-겹) 교차 검증 사용
# 데이터를 k개의 분할(fold, 폴드)로 나누고(일반적ㅇ로 4 또는 5)
# k개의 모델을 각각 만들어 k - 1 개의 세트로 학습시키고 나머지는 1개의 세트로 검증하는 방법
# 모델의 예측도 점수는 k개의 점수의 평균으로 처리
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epochs = 10
all_scores = []
for i in range(k):
    print(i, 'fold')
    # 검증용 데이터 준비 : k번째 분할
    val_data = train_data[i*num_val_samples:(i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples:(i+1) * num_val_samples]
    
    # 학습용 데이터 준비 : 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i+1)*num_val_samples:]], axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i + 1) * num_val_samples:]], axis=0)
    
    # 케라스 모델 구성
    model = build_model()
    
    # 모델 학습
    model.fit(partial_train_data, partial_train_targets,
             epochs = num_epochs, batch_size=1, verbose=1)
    
    # 검증용 데이터로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_mae)

0 fold
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
303/303 [==============================] - 1s 4ms/step - loss: 202.4445 - mean_absolute_error: 10.7736
Epoch 2/10
303/303 [==============================] - 1s 2ms/step - loss: 30.2679 - mean_absolute_error: 3.8343
Epoch 3/10
303/303 [==============================] - 0s 1ms/step - loss: 21.2854 - mean_absolute_error: 3.1729
Epoch 4/10
303/303 [==============================] - 1s 2ms/step - loss: 18.2128 - mean_absolute_error: 2.8948
Epoch 5/10
303/303 [==============================] - 1s 2ms/step - loss: 16.5866 - mean_absolute_error: 2.6596
Epoch 6/10
303/303 [==============================] - 0s 1ms/step - loss: 15.4178 - mean_absolute_error: 2.5639
Epoch 7/10
303/303 [==============================] - 0s 1ms/step - loss: 14.1868 - mean_absolute_error: 2.5340
Epoch 8/10
303/303 [==============================] - 0s 1ms/step - loss: 13.8043 - mea

In [10]:
all_scores

[2.4877246677285374,
 2.5931055947105484,
 2.5399238567541143,
 2.8044408370952794]

In [11]:
np.mean(all_scores)
# 2600달러 정도 차이가 남

2.60629873907212